<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h1 class="list-group-item list-group-item-action active" data-toggle="list" style='background:#005097; border:0' role="tab" aria-controls="home"><center>APRENDIZADO DE MÁQUINA (CIC1205) - Trabalho 2</center></h1>

- Nome completo: MATHEUS SANTOS MELO
- Matrícula: 2430148MCICMA
- [Link para vídeo](https://cefetrjbr-my.sharepoint.com/:v:/g/personal/18888284710_cefet-rj_br/EVKdfxdq6epJjx2ns4rxYtIBLW-OZufNVc10ZvPnMGGFhw?e=x38xWU&nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D)

- Para esse trabalho, utilizei a versão do Python 3.12.4.

## 0. Importação das bibliotecas para todas as questões

In [ ]:
# Manipulação de Dados
import pandas as pd
import numpy as np

# Visualização de Dados
import matplotlib.pyplot as plt
import plotly.subplots as sp
import plotly.graph_objects as go

# Algoritmos de Classificação e Regressão
from sklearn.ensemble import GradientBoostingClassifier

# Pré-processamento e Transformações
from sklearn.preprocessing import OrdinalEncoder

# Avaliação de Modelos
from sklearn.metrics import confusion_matrix, classification_report

# Utilitários
import pickle
import warnings

# Para ignorar mensagens de aviso
warnings.filterwarnings(action='ignore')

# Parâmetro para utilizar no random_state por fins de reprodutibilidade
seed = 123

## 1. Engenharia de Features

- Para essa questão, irei criar alguns atributos derivados que façam sentido e depois irei analisá-los para saber se tem uma relação com a variável-alvo.

#### Descrição dos dados (Fonte: [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds)):

- **price**: price in US dollars ($326--$18,823)

- **carat**: weight of the diamond (0.2--5.01)

- **cut**: quality of the cut (Fair, Good, Very Good, Premium, Ideal)

- **color**: diamond colour, from J (worst) to D (best)

- **clarity**: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

- **x**: length in mm (0--10.74)

- **y**: width in mm (0--58.9)

- **z**: depth in mm (0--31.8)

- **depth**: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

- **table**: width of top of diamond relative to widest point (43--95)

In [ ]:
diamonds = pd.read_csv('../data/diamonds.csv', index_col=0)
diamonds

### Feature 1: Volume do diamante

- Justificativa: O volume é uma medida física tridimensional, e pode representar melhor o "corpo" real do diamante, principalmente quando existem variações no formato que não são bem explicadas só pelo `carat`.

- Variáveis utilizadas: `x`, `y`, `z` 

In [ ]:
diamonds['volume'] = diamonds['x'] * diamonds['y'] * diamonds['z']

In [ ]:
fig = sp.make_subplots(rows=1, cols=2, horizontal_spacing=0.1, subplot_titles=['Scatterplot', 'Heatmap'])

# Primeiro subplot
fig.add_trace(go.Scatter(x= diamonds['volume'], y=diamonds['price'], mode='markers'), row=1, col=1)

# Segundo subplot
fig.add_trace(go.Heatmap(z = diamonds[['volume', 'price']].corr(), zmin=-1, zmax=1, 
                         x = ['volume', 'price'], y = ['volume', 'price'], texttemplate="%{z:.3f}", coloraxis="coloraxis"), row=1, col=2)

# Atualizando layout com rótulos dos eixos e exibição do gráfico
fig.update_layout(xaxis1_title="volume", yaxis1_title="price", height=400, width=1250, coloraxis={'colorscale': 'Turbo', 'cmin': -1, 'cmax': 1}, 
                  title= 'Análise exploratória entre a feature criada e o preço')

fig.show('png') 

- Podemos observar que o processo de Feature Engineering para criação da variável de volume tem uma correlação positiva não-linear alta em relação ao preço do diamante. Ou seja, quanto maior for o volume do diamante, maior o preço, o que faz sentido seguindo a lógica que o preço aumenta conforme o valor do diamanete. 

- O ponto interessante é que como foi imaginado, o valor do diamante aumenta de uma maneira exponencial de acordo com o seu volume, o que pode ser visto a partir do gráfico de dispersão acima.

- Sendo assim, é uma boa variável para ser considerada para o modelo preditivo.

### Features 2 e 3: Quilate com transformação logarítmica e polinomial

- Justificativa: A transformação logarítmica e polinomial de `carat` pode ajudar a linearizar sua relação com o preço, além de possivelmente reduzir o impacto de outliers e capturar eventuais efeitos não lineares do peso sobre o valor do diamante. Criarei as duas features pois minha intenção é testar essa abordagem de transformação tanto de forma logarítimica quanto polinomial e ver qual possui melhor relação com o target.

- Variável utilizada: `carat`

In [ ]:
import math

def apply_log(X):
    return math.log(X, 10) 

def apply_poly(X):
    return X**2

diamonds['log_carat'] = diamonds['carat'].apply(lambda k: apply_log(k))
diamonds['carat²'] = diamonds['carat'].apply(lambda k: apply_poly(k))

In [ ]:
fig = sp.make_subplots(rows=2, cols=2, horizontal_spacing=0.1, subplot_titles=['Scatterplot', 'Heatmap', 'Scatterplot', 'Heatmap'], vertical_spacing=0.25)

# Primeiro subplot
fig.add_trace(go.Scatter(x= diamonds['log_carat'], y=diamonds['price'], mode='markers'), row=1, col=1)

# Segundo subplot
fig.add_trace(go.Heatmap(z = diamonds[['log_carat', 'price']].corr('spearman'), zmin=-1, zmax=1, 
                         x = ['log_carat', 'price'], y = ['log_carat', 'price'], texttemplate="%{z:.3f}", coloraxis="coloraxis"), row=1, col=2)

# Terceiro subplot
fig.add_trace(go.Scatter(x= diamonds['carat²'], y=diamonds['price'], mode='markers'), row=2, col=1)

# Quarto subplot
fig.add_trace(go.Heatmap(z = diamonds[['carat²', 'price']].corr('spearman'), zmin=-1, zmax=1, 
                         x = ['carat²', 'price'], y = ['carat²', 'price'], texttemplate="%{z:.3f}", coloraxis="coloraxis"), row=2, col=2)

# Atualizando layout com rótulos dos eixos e exibição do gráfico
fig.update_layout(xaxis1_title="log_carat", yaxis1_title="price", xaxis3_title="carat²", yaxis3_title="price", 
                  height=800, width=1250, coloraxis={'colorscale': 'Turbo', 'cmin': -1, 'cmax': 1}, 
                  title= 'Análise exploratória entre as features criadas e o preço')

fig.show('png') 

- Acima foram feitas as transformações da variável `carat` por meio da transformação logarítmica e quadrática, gerando duas novas variáveis. Considerando esse contexto, faz sentido que essas variáveis apresentem relações não lineares (como observado nos gráficos de dispersão). Por esse motivo, foi aplicada a correlação de Spearman no heatmap, uma vez que ela é adequada para capturar relações monotônicas, que podem ser não lineares. Diferentemente da correlação de Pearson, que assume uma relação linear e exige que ambas as variáveis possuam distribuição aproximadamente normal.

- A respeito das correlações, observa-se uma correlação positiva, monotônica e alta para ambas as variáveis, assim como foi observado anteriormente com a feature volume. Como ambas as variáveis são transformações diferentes da variável `carat` em escalas distintas, seus valores numéricos no gráfico de dispersão são diferentes, mas a correlação permanece praticamente a mesma, já que ambas medem essencialmente a mesma informação. 

- Portanto, seria interessante considerar uma ou outra na próxima etapa de modelagem preditiva, evitando a inclusão simultânea devido à alta multicolinearidade entre elas.

## 2. Classificação Ordinal Multi-classes

In [ ]:
filename = 'A652.pickle'

f = open(filename, 'rb')

(X_train, y_train, X_val, y_val, X_test, y_test) = pickle.load(f)
print("Shapes: ", X_train.shape, X_val.shape, X_test.shape)

### Configurações experimentais (Pré-processamento)

In [ ]:
# Função para transformar os targets númericos y_* em categórico de acordo com os ranges do enunciado
def numerical_to_categorical(y):
    """    
    Função para categorizar as faixas de valores do target númerico
    
    0           -> 'None'
    (0, 5]      -> 'Weak'
    (5, 25]     -> 'Moderate'
    (25, 50]    -> 'Strong'
    (50, +inf]  -> 'Extreme'
    """

    conditions = [
        y == 0,
        (y > 0) & (y <= 5),
        (y > 5) & (y <= 25),
        (y > 25) & (y <= 50),
        y > 50
    ]

    choices = ['None', 'Weak', 'Moderate', 'Strong', 'Extreme']

    return np.select(conditions, choices, default='Unknown')

In [ ]:
# Transformação dos ranges númericos dos targets de treino, validação e teste em categorias
y_train_cat = numerical_to_categorical(y_train.ravel())
y_test_cat = numerical_to_categorical(y_test.ravel())

print('y_train:\n', pd.Series(y_train_cat).value_counts())
print('\ny_test:\n', pd.Series(y_test_cat).value_counts())

In [ ]:
y_test_cat = np.where(y_test_cat == 'Extreme', 'Strong', y_test_cat)
print('\ny_test:\n', pd.Series(y_test_cat).value_counts())

- Como não há a classe 'Extreme' no conjunto de treino, isso pode gerar problemas na etapa de predição multiclasse ordinal, pois existe um ponto em que é criado um target para comparar 'Target > Classe?' e isso irá gerar um target apenas constituído por zero a partir do 'Strong', pois não há nenhum valor acima dele igual a 1 no conjunto de treino, dando erro no algoritmo do GradientBoostClassifier. Logo, para evitar esse problema, substitui o 'Extreme' para a classe 'Strong'.

In [ ]:
# Define a ordem das categorias e aplica o OrdinalEncoder para codificar as categorias do targets em um númerica ordinal
# O OrdinalEnconder foi aplicado de uma forma que evitasse DataLeakage
categories_order = [['None', 'Weak', 'Moderate', 'Strong', 'Extreme']]
encoder = OrdinalEncoder(categories=categories_order)

y_train_ord = pd.Series(encoder.fit_transform(y_train_cat.reshape(-1, 1)).astype(int).ravel())
y_test_ord = pd.Series(encoder.transform(y_test_cat.reshape(-1, 1)).astype(int).ravel())

print('y_train:\n', y_train_ord.value_counts())
print('\ny_test:\n', y_test_ord.value_counts())

- Podemos observar que a classe majoritária nas duas porções é a 'None' (classe 0), o que significa que a maior parte dos dados representa momentos sem chuva. O desbalanceamento é evidente e pode prejudicar a performance dos modelos de Machine Learning.

- Outro ponto a se notar é que a categoria 'Extreme' (classe 4) é tão rara que não possui registros nas porções de treino, aparecendo apenas duas vezes no conjunto de teste. Isso indica que o modelo provavelmente não terá a oportunidade de aprender sobre essa classe, sendo assim, tenderá a "chutar" um valor para ela.

### Modelagem preditiva Baseline (Multiclassificação normal)

#### Treinamento e predição

In [ ]:
def plot_confusion_matrix_class_report(y_test, y_pred, labels=None, target_names=None):
    """
    
    Função para obter a matriz de confusão e classification_report

    """
    #Confusion metrics of the model built on down-sampled data
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=labels)

    fig, ax = plt.subplots(figsize=(3, 3))
    ax.matshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', color="white" if conf_matrix[i, j] > conf_matrix.max()/2 else "black")
    
    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.show()

    print(classification_report(y_test, y_pred, labels=labels, target_names=target_names))

In [ ]:
# Treinamento do modelo
model = GradientBoostingClassifier(random_state=seed)
model.fit(X_train, y_train_ord)

# Teste
y_pred_test = model.predict(X_test)

y_pred = encoder.inverse_transform(y_pred_test.reshape(-1, 1)).ravel()
y_true = encoder.inverse_transform(y_test_ord.to_numpy().reshape(-1, 1)).ravel()

labels = ['None', 'Weak', 'Moderate', 'Strong']
plot_confusion_matrix_class_report(y_true, y_pred, labels=labels, target_names=labels)

#### Análise dos resultados

- A partir da modelagem baseline de classificação multiclasse, considerando o target como uma variável categórica nominal, o GradientBoostingClassifier adota uma metodologia interna que permite o treinamento com um target não binário. A partir disso, observamos que a acurácia resultante foi bastante alta. No entanto, essa métrica não pode ser considerada confiável, uma vez que há um desbalanceamento acentuado entre as classes, o que favorece o modelo a acertar frequentemente ao simplesmente prever a classe 0. Para uma avaliação mais robusta, será utilizado principalmente o f1-score por classe.

- Considerando que o objetivo é prever se haverá chuva, as classes mais relevantes a serem previstas são da 1 à 4. Essas classes apresentam, teoricamente, uma ordem de importância a ser considerada, sendo cada vez mais interessante prever corretamente as classes superiores, pois representam eventos mais raros. Porém, para essa modelagem baseline isso não foi considerado. No entanto, ess

    Como observado anteriormente, a raridade de classes implica em menos amostras disponíveis para o treinamento do modelo. Logo, o conjunto de treinamento possuiu apenas dois exemplos para a classe 3 e nenhum exemplo para a classe 4, o que justifica o f1-score de 0% para essas duas classes mais raras, pois o modelo não teve informações suficientes (ou nenhuma) para aprender a predizê-las, sofrendo assim com underfitting e um grande desbalanceamento. O mesmo ocorreu em menor grau com a classe 2, que contou com poucos exemplos no treino e teste, resultando em um desempenho igualmente baixo e quase nulo.

    Por fim, a segunda melhor classe em desempenho foi a classe 1, a qual apresentou a segunda maior quantidade de exemplos. O modelo pôde realizar um treinamento mais adequado sobre essa classe, alcançando um F1-score de 41%.

### Modelagem preditiva principal (Multiclassificação ordinal)

#### Treinamento e predição

In [ ]:
def predict_ordinal_proba(X, models):
    """
    Combina as previsões dos modelos binários para estimar as probabilidades de cada classe ordinal.
    """
    k = len(models) + 1  # número total de classes
    n = X.shape[0]       # número de instâncias
    probas = np.zeros((n, k))  # matriz de probabilidades finais

    # Obtemos P(y > i) de cada classificador binário
    prob_gt = [model.predict_proba(X)[:, 1] for model in models]

    # Probabilidade da primeira classe (ex: 'None') é: 1 - P(y > 0)
    probas[:, 0] = 1 - prob_gt[0]

    # Classes intermediárias: P(y = i) = P(y > i−1) * (1 − P(y > i))
    for i in range(1, k - 1):
        probas[:, i] = prob_gt[i - 1] * (1 - prob_gt[i])

    # Última classe é simplesmente P(y > k−2), pois é teoricamente o restante
    probas[:, k - 1] = prob_gt[-1]

    return probas

In [ ]:
k = y_train_ord.nunique()
models = []

# Para cada limiar i (i = 0 até k−2), criamos um modelo que responde: "Classe > i?"
for i in range(k - 1):
    print('Target index:', i)
    print('Valores por classe:\n', y_train_ord.value_counts())
    y_binary = (y_train_ord > i).astype(int)  # binariza o target: 1 se classe > i, senão 0

    print('Classe > i?:\n', y_binary.value_counts())
    clf = GradientBoostingClassifier(random_state=seed)
    clf.fit(X_train, y_binary)  # Treina modelo binário para esse limiar
    models.append(clf)
    print('--------------\n')

# Estima as probabilidades finais por classe
probas_test = predict_ordinal_proba(X_test, models)

# A classe final é aquela com maior probabilidade (argmax)
y_pred_ord = np.argmax(probas_test, axis=1)

# Decodifica os rótulos numéricos de volta para nomes de classe
y_pred_labels = encoder.inverse_transform(y_pred_ord.reshape(-1, 1)).ravel()
y_true_labels = encoder.inverse_transform(y_test_ord.to_numpy().reshape(-1, 1)).ravel()

labels = ['None', 'Weak', 'Moderate', 'Strong']
plot_confusion_matrix_class_report(y_true_labels, y_pred_labels, labels=labels, target_names=labels)

#### Análise dos resultados

- Por meio da metodologia de classificação ordinal multiclasse abordada no artigo, foi realizada a modelagem para cada dataset associado a uma tarefa binária do tipo "Classe > i?", sendo atribuído o valor 1 nos casos em que a classe está acima na hierarquia ordinal, e 0 caso contrário. Esse processo foi repetido para cada uma das classes, exceto para a última (no caso, 'Strong') pois não há valores superiores a ela, então não faria sentido. A partir dessa modelagem, foi possível estimar as probabilidades para cada uma das classes, sendo escolhida como classificação final aquela com maior probabilidade entre todas, para cada exemplo no conjunto de teste.

- Em relação aos resultados metrificados pelo f1-score principalmente, observa-se que, assim como na abordagem tradicional (baseline), o desbalanceamento dos dados continua sendo um problema predominante, fazendo com que a classe 'None' tenha desempenho significativamente superior, enquanto as demais apresentem métricas mais baixas, como é o caso da classe 'Strong', que teve apenas 2 exemplos no conjunto de treinamento.

    Considerando o respeito à ordem entre as classes proporcionado por essa metodologia alternativa, percebe-se que, apesar do desbalanceamento favorecer classificações para a classe majoritária, houve uma melhora no f1-score da classe 'Weak', passando de 41% para 46% (um aumento de 5%), e da classe 'Moderate', de 6% para 7% (um aumento de 1%). Esses resultados indicam que a segunda abordagem é útil para capturar nuances nos dados ordinais que não foram percebidas no baseline, devido ao fato de este tratar o target como se fosse uma variável nominal.

## 3. SHAP Values

## 4. Redução de Dimensionalidade

## 5. Predição Conforme